In [1]:
# This file is made to scrap Ireland information on scams
# importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import date

In [2]:
html_response_check = requests.get('Deleted for security reasons')
print(html_response_check)

<Response [200]>


### Current script for scrapping ALL links, dates, and titles.

In [3]:
# Function to scrape links, dates, and titles from a given page
def scrape_links(page_number):
    url = f'Deleted for security reasons{page_number}'
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    articles = soup.find_all('div', class_='spotlight-content')
    
    link_details = []
    for article in articles:
        try:
            date = article.find('span', class_='spotlight-label').get_text(strip=True)
        except AttributeError:
            date = None
        
        try:
            link = article.find('a', href=True)['href']
        except TypeError:
            link = None
        
        try:
            title = article.find('h4', class_='spotlight-title').get_text(strip=True)
        except AttributeError:
            title = None
        
        if link:  # Only append if the link is found
            link_details.append({'Date': date, 'Link': link, 'Title': title})
    
    return link_details

# List to store all the links and their additional information
all_links_details = []

# Loop through each page and collect hrefs, dates, and titles
for page in range(1, 3): # in range last number is excluding
    print(f"Scraping page {page}...")
    page_details = scrape_links(page)
    all_links_details.extend(page_details)

# Create a DataFrame from the collected details
df = pd.DataFrame(all_links_details)

today = date.today() # used to get todays date 

# Save the DataFrame to an Excel file
df.to_excel(f'scraped_links_details_{today}.xlsx', index=False)

Scraping page 1...
Scraping page 2...


### Current script for scrapping ALL new links inside cases.

In [ ]:
today = date.today()

def scrape_website(link):
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    
    website = None
    rows = soup.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:
            key = cols[0].get_text(strip=True).replace(':', '')
            value = cols[1].get_text(strip=True)
            if key == 'Website':
                website = value
                break
    
    return website

# Read the initial links from the 'scraped_links_details.xlsx' file
df_links = pd.read_excel(f'scraped_links_details_{today}.xlsx')

# List to store the website details
website_details = []

# Loop through each link and collect the website details
for index, row in df_links.iterrows():
    link = row['Link']
    website = scrape_website(link)
    website_details.append(website)
    print(f"Scraped website for {link}")

# Add the website details to the original DataFrame
df_links['Website'] = website_details


# Save the combined DataFrame to a new Excel file
df_links.to_excel(f'website_details_{today}.xlsx', index=False)

### Current script for scrapping ALL old links inside cases. (old formatted cases)

In [ ]:
def unordinary_links(link):
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    
    text = soup.get_text()
    first_link = None
    
    # Regular expression to find the first URL in the text
    match = re.search(r'http[s]?://\S+', text)
    if match:
        first_link = match.group(0)
    
    return first_link

# Read the initial links from the 'website_details.xlsx' file
df_links = pd.read_excel('website_details.xlsx')

# Loop through each link and update the "Website" column if it is empty
for index, row in df_links.iterrows():
    if pd.isna(row['Website']) or row['Website'] == '':
        link = row['Link']
        website = unordinary_links(link)
        df_links.at[index, 'Website'] = website
        print(f"Scraped first link for {link}")

# Save the updated DataFrame to a new Excel file
df_links.to_excel('updated_website_details.xlsx', index=False)